<a href="https://colab.research.google.com/github/searchs/analytics/blob/master/prediction_one_service.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import socket 
hostname = socket.gethostname()

In [ ]:
print(hostname)

9e21871c6c50


In [ ]:
ip_add = socket.gethostbyname(hostname)
ip_add

'172.28.0.2'

In [ ]:
from flask import Flask
app = Flask(__name__)

In [ ]:
@app.route('/')
def allo():
  return "Allo Ola!"

In [ ]:
import  threading

In [ ]:
# target=app.run()
flask_thread = threading.Thread(target=app.run, kwargs={'host': '0.0.0.0', 'port': 80})

In [ ]:
flask_thread.start()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production


In [ ]:
import requests
r = requests.get("http://172.28.0.2")

   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:80/ (Press CTRL+C to quit)
172.28.0.2 - - [22/Jun/2021 07:06:58] "GET / HTTP/1.1" 200 -


In [ ]:
r.content

b'Allo Ola!'

In [ ]:
r.text

'Allo Ola!'

In [ ]:
r = requests.post('http://172.28.0.2')

172.28.0.2 - - [22/Jun/2021 07:06:58] "POST / HTTP/1.1" 405 -


In [ ]:
@app.route('/foo', methods=['GET','POST'])
def foo():
  return "Allo Foo!"

In [ ]:
url = 'http://172.28.0.2/'

In [ ]:
r_get = requests.get(url + 'foo')
r_get.text

172.28.0.2 - - [22/Jun/2021 07:06:58] "GET /foo HTTP/1.1" 200 -


'Allo Foo!'

In [ ]:
r_post = requests.get(url + 'foo')
r_post.text

172.28.0.2 - - [22/Jun/2021 07:06:58] "GET /foo HTTP/1.1" 200 -


'Allo Foo!'

In [ ]:
from flask import jsonify, request
import json

@app.route('/display', methods=['POST'])
def print_item():
  data = request.get_json()
  return jsonify(data)

In [ ]:
data = ['Australia', 'France', 'China']

In [ ]:
j_data = json.dumps(data)

In [ ]:
j_data

'["Australia", "France", "China"]'

In [ ]:
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

In [ ]:
r = requests.post(url + 'display',data=j_data, headers=headers)

172.28.0.2 - - [22/Jun/2021 07:06:58] "POST /display HTTP/1.1" 200 -


In [ ]:
r.text

'["Australia","France","China"]\n'

In [ ]:
@app.route('/empty', methods=['POST'])
def check_empty():
  data = request.get_json()
  return jsonify(not data)

In [ ]:
empty_json = json.dumps([])

In [ ]:
r_empty = requests.post(url +'/empty', data=empty_json, headers=headers)

172.28.0.2 - - [22/Jun/2021 07:06:58] "POST //empty HTTP/1.1" 200 -


In [ ]:
r_empty.text

'true\n'

In [ ]:
not_empty_json = json.dumps(['Data Science', 'is', \
                             'so', 'cool', '!'])

In [ ]:
r_not_empty = requests.post(url + '/empty', \
                            data=not_empty_json, headers=headers)
r_not_empty.text

172.28.0.2 - - [22/Jun/2021 07:06:58] "POST //empty HTTP/1.1" 200 -


'false\n'

In [ ]:
# Create a Model 
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [ ]:
file_url = 'https://raw.githubusercontent.com/PacktWorkshops/'\
           'The-Data-Science-Workshop/master/'\
           'Chapter03/bank-full.csv'
df = pd.read_csv(file_url, sep=';')

In [ ]:
# Extract response 
y = df.pop('y')

In [ ]:
df_dummies = pd.get_dummies(df)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_dummies, y, test_size=0.33, random_state=42)

In [ ]:
rf_model = RandomForestClassifier(random_state=8)
rf_model.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=8, verbose=0,
                       warm_start=False)

In [ ]:
rf_model.predict(X_test)

array(['no', 'no', 'no', ..., 'no', 'no', 'yes'], dtype=object)

In [ ]:
rf_model.predict([X_test.iloc[3776,]])

array(['no'], dtype=object)

In [ ]:
import joblib
joblib.dump(rf_model, "model.pkl")

['model.pkl']

In [ ]:
saved_model = joblib.load("model.pkl")

In [ ]:
# Make prediction
saved_model.predict([X_test.iloc[3776,]])

array(['no'], dtype=object)

In [ ]:
import numpy as np

@app.route('/predict', methods=['POST'])
def rf_predict():
  data = request.get_json()
  prediction = saved_model.predict(data)
  str_pred = np.array2string(prediction)
  return jsonify(str_pred)

In [ ]:

record = X_test.iloc[3776,].to_list()
record [:7]

[36, 229, 28, 258, 2, -1, 0]

In [ ]:
j_data = json.dumps([record])

In [ ]:
r = requests.post(url +"/predict", data=j_data, headers=headers)
r.text

172.28.0.2 - - [22/Jun/2021 07:07:04] "POST //predict HTTP/1.1" 200 -


'"[\'no\']"\n'

In [ ]:
import os
import sys
import platform

In [ ]:
print(platform.uname())

uname_result(system='Linux', node='9e21871c6c50', release='5.4.104+', version='#1 SMP Sat Jun 5 09:50:34 PDT 2021', machine='x86_64', processor='x86_64')
